In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### These data set about covid_19. The dataset contains the many things . through these data we are anlysing the rise in cases ,rise in death rate ,and different aged group who were covid postive and how many were discharged, if any patients is suffering from any other diseases for example diabetes,breathing problems and many others diseases .

# first import all necesseary dictionary

In [ ]:
import pandas as pd
import datetime
from datetime import date
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
import fbprophet 
import numpy as np
%matplotlib inline
import os
import seaborn as sns
import lightgbm as lgb
import time
import matplotlib.dates as mdates
import datetime as dt

from tqdm import tqdm
from random import choice

## After importing all important dictionary next we will be importing csv file or we will reading csv file

In [ ]:
df_data = pd.read_csv("../input/project-by-jyothi-on-covid-19/owid-covid-data (1).csv")

# finding shape, size, type of data set 

In [ ]:
type(df_data)

In [ ]:
df_data

In [ ]:
df_data.shape

In [ ]:
df_data.size

# countint notnull values

In [ ]:
df_data.notnull().count()

# to see top n rows

In [ ]:

df_data.head(50)

# to see bottom n rows

In [ ]:

df_data.tail(12)

In [ ]:
df_data.info()

In [ ]:
df_data.describe()

# fing unique values from the different columns from the data 

In [ ]:

df_data['total_cases'].unique()

In [ ]:



df_data['new_cases'].unique()




In [ ]:
df_data['new_cases_smoothed'].unique()



In [ ]:
df_data['total_deaths'].unique()



In [ ]:
df_data['new_deaths'].unique()



In [ ]:
df_data['new_deaths_smoothed'].unique()



In [ ]:
df_data['total_cases_per_million'].unique()



In [ ]:
df_data['new_cases_per_million'].unique()



In [ ]:
df_data['new_cases_smoothed_per_million'].unique()

In [ ]:
df_data['total_deaths_per_million'].unique()



In [ ]:

df_data['aged_70_older'].unique()


In [ ]:
df_data['gdp_per_capita'].unique()


In [ ]:
df_data['extreme_poverty'].unique()


In [ ]:
df_data['cardiovasc_death_rate'].unique()


In [ ]:
df_data['female_smokers'].unique()


In [ ]:
df_data['male_smokers'].unique()


In [ ]:
df_data['handwashing_facilities'].unique()


In [ ]:
df_data['diabetes_prevalence'].unique()

In [ ]:
df_data['hospital_beds_per_thousand'].unique()





In [ ]:
df_data['life_expectancy'].unique()

# by using max and min function

In [ ]:
df_data['life_expectancy'].max()

In [ ]:
df_data['life_expectancy'].min()

In [ ]:
df_data['life_expectancy'].median()

# shorting required data from data set

hear we are shoring required data which help us to anlysise the data proprely

In [ ]:
# for data exploration we only need 5 columns:,Country,Date,Total cases,Age,death 
Data = pd.DataFrame({'Country':df_data['location'],'Date':df_data['date'],'Cases':df_data['total_cases'],'Age':df_data['median_age'],'Death':df_data['total_deaths']})

# Replacing NaN with 0

In [ ]:
Data.fillna(0)

In [ ]:

Data['Date'] = pd.to_datetime(Data['Date']).dt.strftime('%d-%m-%Y')


# total cases reportaded and medain aged group cases 
# on any date

In [ ]:
data = df_data[(df_data['location']=='World')&(df_data['date']=='2020-03-01')]
total_cases = data['total_cases'].max()
median_aged= df_data['median_age'].max()
print("Total cases reported until 1st of march 2020 worldwide is {}\n".format(total_cases))
print("Total cases of median age grouped 1st of March 2020 worldwide is {}".format(median_aged))

# by using 'by_date'function we are grouping data on bases of total death and number of death on a date

In [ ]:
def find_contry(contry,date):
    find_row = Data[(Data['Country']==contry)&(Data['Date']==date)]
    deth = find_row['Death'].max()
    case = find_row['Cases'].max()
    print("{} has reported {} covid 19 cases and {} deaths due to covid 19 on {}".format(contry,case,deth,date))

In [ ]:
find_contry('India','10-08-2020')

In [ ]:
def by_date(date):
    df_by_date = Data[(Data['Date']==date)&(Data['Country']!='International')&(Data['Country']!='World')].sort_values(by='Country').reset_index().drop(columns='index')
    print(df_by_date)

In [ ]:
by_date('10-08-2020')

In [ ]:
def find_contry(contry,date):
    find_row = Data[(Data['Country']==contry)&(Data['Date']==date)]
    deth = find_row['Death'].max()
    case = find_row['Cases'].max()
    print("{} has reported {} covid 19 cases and {} deaths due to covid 19 on {}".format(contry,case,deth,date))

In [ ]:
find_contry('China','10-03-2020')

# plotting of data on different bases

In [ ]:

covid_19 = pd.DataFrame({'Country':df_data['location'],'Date':df_data['date'],
                         'Death':df_data['total_deaths'],
                         'Cases':df_data['total_cases'],'Age':df_data['median_age'],
                         'Aged':df_data['aged_65_older'],
                       'New cases':df_data['new_cases'],'New death':df_data['new_deaths']})
covid_19['month'] = pd.to_datetime(covid_19['Date'],format='%Y-%m-%d')
covid_19['Date'] = pd.to_datetime(covid_19['Date'],format='%Y-%m-%d')
covid_19 = covid_19.fillna(0)

In [ ]:
covid_19

# total cases vs. Date graph

In [ ]:

_, ax = plt.subplots(1,1, figsize=(25,9))
covid_19_cont = covid_19[(covid_19['Country']=='India')&(covid_19['Date']!='2020-8-29')]
ax.plot(covid_19_cont.month,
        covid_19_cont.Cases,  
        linestyle='',
        marker='d'
        
       )
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total covid cases vs date')

### from the above graph we can predict two things
1. as from july there has been tremendous increase in cases till august the cases have crossed around 3,50,00,000
2. we can also see that from januery to july there was slow increase in cases

# daily reported caes vs death cases

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont.month,
        covid_19_cont['New cases'],  
        linestyle='',
        marker='o'
        
       )
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total New cases vs date')
plt.grid()

# compersion between daily reported covid cases vs. death reported cases 

In [ ]:
_, ax = plt.subplots(1,1, figsize=(19,9))
ax.plot(covid_19_cont.month,
        covid_19_cont['New death'],   
        label='daeth',
        color='c'
       )
ax.plot(covid_19_cont.month,
        covid_19_cont['New cases'], 
        label='cases',
        color='r'
       )
ax.legend()
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total Covid deaths vs date')
plt.grid()

### Though there is tremendous increases in covid cases we can aslo there is less amount od daeth due to covid

# Daily reporte cases vs deaths

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New cases'],
        covid_19_cont['New death'],  
        linestyle='',
        marker='d',
        color='k'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('Death')
plt.title('total Covid deaths vs date')

## here we are plotting data on bases age

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New cases'],
        covid_19_cont['Age'],  
        linestyle='',
        marker='d',
        color='k'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('AGE')
plt.title('total Covid deaths vs date')

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New death'],
        covid_19_cont['Age'],  
        linestyle='',
        marker='d',
        color='C'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('AGE')
plt.title('total Covid deaths vs date')

# here we are plotting data for china

how much cases were reported in chian on an particular date

In [ ]:
_, ax = plt.subplots(1,1, figsize=(25,9))
covid_19_cont = covid_19[(covid_19['Country']=='China')&(covid_19['Date']!='2020-3-29')]
ax.plot(covid_19_cont.month,
        covid_19_cont.Cases,  
        linestyle='',
        marker='d'
        
       )
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total covid cases vs date')


daily reported caes vs death cases







In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont.month,
        covid_19_cont['New cases'],  
        linestyle='',
        marker='o'
        
       )
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total New cases vs date')
plt.grid()

## compersion between daily reported covid cases vs. death reported cases

In [ ]:
_, ax = plt.subplots(1,1, figsize=(19,9))
ax.plot(covid_19_cont.month,
        covid_19_cont['New death'],   
        label='daeth',
        color='c'
       )
ax.plot(covid_19_cont.month,
        covid_19_cont['New cases'], 
        label='cases',
        color='k'
       )
ax.legend()
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total Covid deaths vs date')
plt.grid()

# Daily reporte cases vs deaths for china

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New cases'],
        covid_19_cont['New death'],  
        linestyle='',
        marker='o',
        color='r'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('Death')
plt.title('total Covid deaths vs date')

# these plt shows us how many case were reported under age of 65

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New cases'],
        covid_19_cont['Aged'],  
        linestyle='',
        marker='d',
        color='C'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('AGE')
plt.title('total Covid deaths vs date')

# Here we are ploting data for Australia

In [ ]:
_, ax = plt.subplots(1,1, figsize=(25,9))
covid_19_cont = covid_19[(covid_19['Country']=='Australia')&(covid_19['Date']!='2020-8-1')]
ax.plot(covid_19_cont.month,
        covid_19_cont.Cases,  
        linestyle='',
        marker='d'
        
       )
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total covid cases vs date')


# daily reported caes vs death cases

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont.month,
        covid_19_cont['New cases'],  
        linestyle='',
        marker='o'
        
       )
plt.xlabel('Date')
plt.ylabel('Cases')
plt.title('total New cases vs date')
plt.grid()

## compersion between daily reported covid cases vs. death reported cases in Australia

In [ ]:
_, ax = plt.subplots(1,1, figsize=(19,9))
ax.plot(covid_19_cont.month,
        covid_19_cont['New death'],   
        label='death',
        color='c'
       )
ax.plot(covid_19_cont.month,
        covid_19_cont['New cases'], 
        label='cases',
        color='k'
       )
ax.legend()
plt.xlabel('Death')
plt.ylabel('Cases')
plt.title('total Covid deaths vs date')
plt.grid()

# Daily reporte cases vs deaths for Australia

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New cases'],
        covid_19_cont['New death'],  
        linestyle='',
        marker='d',
        color='g'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('Death')
plt.title('total Covid deaths vs date')

# these plt shows us how many case were reported under age of 65

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New cases'],
        covid_19_cont['Aged'],  
        linestyle='',
        marker='d',
        color='C'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('AGE')
plt.title('total Covid deaths vs date')

# the next we are plotting data on bases of age group from 18 or more than 18

In [ ]:
_, ax = plt.subplots(1,1, figsize=(20,9))
ax.plot(covid_19_cont['New cases'],
        covid_19_cont['Age'],  
        linestyle='',
        marker='o',
        color='r'
       )
plt.grid()
plt.xlabel('Cases')
plt.ylabel('AGE')
plt.title('total Covid deaths vs date')

# BY using fbprophet we can make future preductions that how cases may rise or may not rise 

# Now we will working on fbprophet which will help us to forecast the dataset

#### for using using fbProphet first we will import the fbprophet  
as we have already imported all necessary dictionary 

In [ ]:
df_data.head()

In [ ]:
df_data.tail()

In [ ]:
covid_19 = pd.DataFrame({'Country':df_data['location'],'Date':df_data['date'],'cases':df_data['total_cases']})
covid_19

In [ ]:
day_summary = df_data.groupby('date').agg(total_cases = ('total_cases', sum),
                                        
                                          date = ('date',max)
                                         )
                                        
day_summary

# Prophet requires columns ds (Date) and y (value)

In [ ]:
day_summary = day_summary.rename(columns={'date': 'ds', 'total_cases': 'y'})
day_summary

# Makeing the prophet model and fitting the data

In [ ]:
gm_prophet = fbprophet.Prophet(changepoint_prior_scale=0.25) # the parameter defines how tightly you want to fit your model

gm_prophet.fit(day_summary)

In [ ]:

# Makeing a future dataframe for 1 month
gm_forecast = gm_prophet.make_future_dataframe(periods=99 , freq='h')

# Make predictions
gm_forecast = gm_prophet.predict(gm_forecast)

In [ ]:
gm_prophet.plot(gm_forecast, xlabel = 'Date', ylabel = 'total_cases')
plt.title('Total covid cases')

In [ ]:
summary = df_data.groupby('location').agg(total_cases = ('total_cases', max),
                                        
                                          location= ('location',max)
                                         )
                                        
summary

In [ ]:
summary = summary.rename(columns={'location': 'y', 'total_cases': 'ds'})
summary

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
split =13
split_index =len(df_data)-split
train=df_data[0:split_index]
test=df_data[split_index:]


In [ ]:
train

In [ ]:
test

In [ ]:
df_data['date'] = pd.to_datetime(df_data['date'])
df_data['date'] = pd.to_datetime(df_data['date'])

In [ ]:
df_data.info()

In [ ]:
df_data['year'] = df_data['date'].dt.year
df_data['month'] = df_data['date'].dt.month
df_data['week'] = df_data['date'].dt.week
df_data['day'] = df_data['date'].dt.day
df_data['dayofweek'] = df_data['date'].dt.dayofweek
#df = df.drop('date', axis=1)
df_data.head()

In [ ]:
corr = df_data.corr()
sns.heatmap(corr, cmap='coolwarm_r', annot_kws={'size':50})

In [ ]:
sense1_renamed = df_data.rename(columns={'date':'ds', 'total_cases':'y'})

In [ ]:
sense1_renamed

In [ ]:
# Makeing the  prophet model and fitting  the data
gm_prophet = fbprophet.Prophet(changepoint_prior_scale=0.25) # the parameter defines how tightly you want to fit your model

gm_prophet.fit(sense1_renamed)


In [ ]:
# Makeing a future dataframe for 4 days
gm_forecast = gm_prophet.make_future_dataframe(periods=90, freq='h')

# Make predictions
gm_forecast = gm_prophet.predict(gm_forecast)

In [ ]:
gm_prophet.plot(gm_forecast, xlabel = 'date', ylabel = 'total_cases')
plt.title('Total covid cases')

In [ ]:
# Make the prophet model and fit on the data
gm_prophet = fbprophet.Prophet(changepoint_prior_scale=0.5) # the parameter defines how tightly you want to fit your model

gm_prophet.fit(sense1_renamed)

In [ ]:
# Make a future dataframe for 4 days
gm_forecast = gm_prophet.make_future_dataframe(periods=96, freq='h')

# Make predictions
gm_forecast = gm_prophet.predict(gm_forecast)
gm_prophet.plot(gm_forecast, xlabel = 'Date', ylabel = 'total_cases')
plt.title('Total covid cases')
#Text(0.5, 1.0, 'Irradiation Prediction')

In [ ]:
gm_prophet.plot_components(gm_forecast);

In [ ]:
gm_forecast

In [ ]:
df_data.fillna(0)   

# Creating  modles for forecasting

## slicing of data

In [ ]:
covid_india = df_data[(df_data['location']=='India')&(df_data['date']!='2020-8-25')]
covid_model = covid_india[['total_cases','date']]
covid_model.reset_index(inplace= True)
covid_model = covid_model.drop(columns='index')
covid_model['date'] = pd.to_datetime(covid_model['date'], format='%Y-%m-%d')
covid_model.index = covid_model.date
covid_model = covid_model.drop(columns='date')
covid_model = covid_model.fillna(0)

train,test = train_test_split(covid_model, test_size=.3, shuffle=False)


# Creating some functions

In [ ]:
test.isnull()

In [ ]:
covid_china = df_data[(df_data['location']=='China')&(df_data['date']!='2020-3-22')]
covid_model = covid_china[['total_cases','date']]
covid_model.reset_index(inplace= True)
covid_model = covid_model.drop(columns='index')
covid_model['date'] = pd.to_datetime(covid_model['date'], format='%Y-%m-%d')
covid_model.index = covid_model.date
covid_model = covid_model.drop(columns='date')
covid_model = covid_model.fillna(0)

train,test = train_test_split(covid_model, test_size=.3, shuffle=False)


In [ ]:
test.isnull()

# using ggplot to plot the data

## i do not know about ggplot i have tried to plot it on different accepects 

In [ ]:
pip install pandas plotnine

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from plotnine import *
%matplotlib inline
df_data = pd.read_csv('../input/project-by-jyothi-on-covid-19/owid-covid-data (1).csv').dropna()

In [ ]:
ggplot(df_data, aes(x='total_cases', y='median_age')) + geom_point()

In [ ]:
ggplot(df_data, aes(x='total_cases', y='median_age',
    size = 'location')) + geom_point()

In [ ]:
ggplot(df_data, aes(x='total_cases', y='aged_65_older', 
    size = 'location', color = 'new_cases')) + geom_point()

In [ ]:
ggplot(df_data, aes(x='median_age')) + \
    geom_bar(stat = 'count')

In [ ]:
ggplot(df_data, aes(x='aged_65_older')) + \
    geom_bar(stat = 'count')

In [ ]:
ggplot(df_data, aes(x='location', fill = 'aged_65_older')) + \
    geom_bar(stat = 'count')

# More geom types

In [ ]:

ggplot(df_data, aes(x='date', y='total_cases')) + \
    geom_boxplot()

In [ ]:
df_data['date'] = df_data['total_cases'].astype("category")

ggplot(df_data, aes(x='date', y='total_cases')) + \
    geom_boxplot()


You'll notice the x-axis labels are overlapped. To flip them 90-degrees we can apply a theme so they look less cluttered. We will revisit themes later.

In [ ]:
ggplot(df_data, aes(x='new_cases', y='total_cases')) + \
    geom_boxplot() + \
    theme(axis_text_x = element_text(angle=90, hjust=1))

# To save some typing, let's define this x-axis label rotating theme as a short variable name that we can reuse:


# Faceting
### ggplot has a special technique called faceting that allows to split one plot into multiple plots based on a factor included in the dataset. We will use it to make one plot for a time series for each column.

In [ ]:
flip_xlabels = theme(axis_text_x = element_text(angle=90, hjust=1))

In [ ]:
ggplot(df_data, aes(x='new_cases', y='total_cases')) + \
    geom_violin() + \
    flip_xlabels

In [ ]:
ggplot(df_data, aes(x='new_cases', y='total_cases')) + \
    geom_boxplot() + \
    facet_wrap(['date']) + \
    flip_xlabels + \
    theme(axis_text_x = element_text(size=6))

In [ ]:
ggplot(df_data, aes(x='new_cases', y='total_cases')) + \
    geom_boxplot() + \
    theme(axis_text_x = element_text(size=4)) + \
    facet_wrap(['date']) + \
    flip_xlabels